# Refining raw data
- 평점을 10개 이하로 받은 영화를 제거하여 새로운 데이터를 생성

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from pprint import pprint

## 1. DATA LOAD

#### sqlalchemy를 이용하여 서버에 있는 data를 dataframe으로 가져옴

In [3]:
engine = create_engine('mysql://{}:{}@{}/{}?charset=utf8'.format(user, pwd, server, db_name), convert_unicode = True)
connect = engine.connect()

raw_data = pd.read_sql_table('data_set', connect)
raw_data.head(2)

c:\Python36\lib\site-packages\sqlalchemy\dialects\mysql\base.py:1569: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  cursor.execute('SELECT @@tx_isolation')


,user,title,genre,score
0,tnen,끝까지 간다,범죄,10
1,tnen,토르: 라그나로크,액션,10


#### COPY DATA

In [22]:
copy_data = raw_data.copy()
copy_data.tail(2)

,user,title,genre,score
90150,ysd5,주토피아,애니메이션,10
90151,ysd5,인턴,코미디,10


## 2. RAW DATA 탐색

- 90151개의 row와 4개의 column이 존재
- 1. user : 1837명
  2. movie(title) : 7262개
  3. genre : 82개
  4. score : 1~10점

In [5]:
copy_data.tail(2)

,user,title,genre,score
90150,ysd5,주토피아,애니메이션,10
90151,ysd5,인턴,코미디,10


In [24]:
# null값 확인
copy_data.isnull().sum()

user     0
title    0
genre    0
score    0
dtype: int64

In [51]:
len_user = len(copy_data['user'].value_counts())
len_title = len(copy_data['title'].value_counts())
len_genre = len(copy_data['genre'].value_counts())

print('<< Before refining data >>')
print('user의 수는 {}\ntitle의 수는 {}\ngenre의 수는 {}'.format(len_user,len_title,len_genre))

<< Before refining data >>
user의 수는 1837
title의 수는 7262
genre의 수는 82


## 3. Refining DATA

#### 1. SETP1 : 10개 이하의 평점을 받은 영화를 제외한다.

In [12]:
# 각 영화가 몇개의 평점을 받았는지 확인
title_number_df = copy_data.groupby('title').count().sort_values('user')
title_number_df.head(2)

,user,genre,score
title,,,
수호전지영웅본색,1,1,1
배꼽,1,1,1


In [25]:
# 10개 이하의 평점을 받는 영화 데이터프레임을 만듬
title_under_10 = title_number_df[title_number_df.user <= 10]

# 10개 이하의 평점을 받은 영화 제목을 리스트로 담음
title_under_10_list = list(title_under_10.index)

print(len(title_under_10_list))
print(title_under_10_list[0])
print(title_under_10_list[5668])
title_under_10.tail(2)

5669
수호전지영웅본색
쉬즈 더 맨


,user,genre,score
title,,,
드라큐라,10,10,10
쉬즈 더 맨,10,10,10


In [26]:
%%time
# 평점을 10명 이하에게 받은 영화를 가지고 있는 row 제거
for title in title_under_10_list:
    copy_data = copy_data[copy_data['title'] != title]

Wall time: 2min 51s


In [29]:
print(len(copy_data['title'].value_counts()))
copy_data.tail(3)

1593


,user,title,genre,score
90149,ysd5,탐정 홍길동: 사라진 마을,드라마,9
90150,ysd5,주토피아,애니메이션,10
90151,ysd5,인턴,코미디,10


In [33]:
# index를 relabel한 새로운 data frame 생성
new_data = copy_data.reset_index(drop=True)
new_data.tail(2)

,user,title,genre,score
73458,ysd5,주토피아,애니메이션,10
73459,ysd5,인턴,코미디,10


## 4. NEW DATA 탐색

- 90151개의 row와 4개의 column이 존재
- 1. user : 1837명
  2. movie(title) : 1593개
  3. genre : 27개
  4. score : 1~10점

In [36]:
# null값 확인
new_data.isnull().sum()

user     0
title    0
genre    0
score    0
dtype: int64

In [37]:
len_user = len(new_data['user'].value_counts())
len_title = len(new_data['title'].value_counts())
len_genre = len(new_data['genre'].value_counts())

print('<< Before refining data >>')
print('user의 수는 {}\ntitle의 수는 {}\ngenre의 수는 {}'.format(len_user,len_title,len_genre))

<< Before refining data >>
user의 수는 1837
title의 수는 1593
genre의 수는 27


In [42]:
new_data.score.value_counts()

10    25209
8     11054
9      9940
7      8147
1      6785
6      5097
5      3294
4      1597
3      1230
2      1107
Name: score, dtype: int64